In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

In [3]:
df=pd.read_csv('/kaggle/input/fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
#drop unnecessary columns
df=df.drop(columns=['id','title','author'],axis=1)

In [5]:
#drop null values
df=df.dropna(axis=0)

In [6]:
from nltk.corpus import stopwords

In [7]:
# creating list of possible stopwords from nltk library
stop=stopwords.words('english')

def cleanText(txt):
    #lowercain
    txt=txt.lower()
    #removing stopwords
    txt=' '.join([word for word in txt.split() if word not in (stop)])
    #removing non-alphabetic characters
    txt=re.sub('[^A-Za-z0-9\s]',' ',txt)
    txt=re.sub('\s',' ',txt)
    return txt

In [8]:
#applying text cleaning function to clean final_news
df['text']=df['text'].apply(cleanText)
df.head()

,text,label
0,house dem aide didn t even see comey s letter...,1
1,ever get feeling life circles roundabout rathe...,0
2,truth might get fired october 29 2016 tension...,1
3,videos 15 civilians killed single us airstrike...,1
4,print iranian woman sentenced six years prison...,1


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-05-19 08:05:34.150480: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 08:05:34.150611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 08:05:34.316675: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
# tokenize 
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['text'])

word_index=tokenizer.word_index

vocab_size=len(word_index)

In [11]:
sent_length=500

sequences=tokenizer.texts_to_sequences(df['text'])
padded_seq=pad_sequences(sequences,maxlen=sent_length,padding='post',truncating='post')

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(padded_seq,df['label'],test_size=0.20,random_state=42,)

In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split


# Example vocab_size definition (should be obtained from your tokenizer)
vocab_size = 10000  # Replace with the actual vocabulary size from your tokenizer
embedding_vector_features = 40
sent_length = 500  # Should match the input length used in padding

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_seq, df['label'], test_size=0.20, random_state=42)

# Define and build the CNN-LSTM model
model = Sequential()

# Add embedding layer
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_vector_features, input_length=sent_length))

# Add Conv1D layer
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))

# Add MaxPooling1D layer
model.add(MaxPooling1D(pool_size=2))

# Add Bidirectional LSTM layer
model.add(Bidirectional(LSTM(100)))

# Add Dropout layer
model.add(Dropout(0.3))

# Add Dense output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.build(input_shape=(None, sent_length))
print(model.summary())

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 40)        │       400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 500, 32)        │         3,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 250, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 200)            │       106,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 510,473 (1.95 MB)

 Trainable params: 510,473 (1.95 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 108s 396ms/step - accuracy: 0.7724 - loss: 0.4229 - val_accuracy: 0.9068 - val_loss: 0.2375
Epoch 2/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 141s 392ms/step - accuracy: 0.9774 - loss: 0.0772 - val_accuracy: 0.9564 - val_loss: 0.1341
Epoch 3/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 101s 389ms/step - accuracy: 0.9899 - loss: 0.0343 - val_accuracy: 0.9523 - val_loss: 0.1421
Epoch 4/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 143s 392ms/step - accuracy: 0.9948 - loss: 0.0197 - val_accuracy: 0.9562 - val_loss: 0.1483
Epoch 5/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 142s 394ms/step - accuracy: 0.9949 - loss: 0.0174 - val_accuracy: 0.9465 - val_loss: 0.2002
Epoch 6/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 141s 392ms/step - accuracy: 0.9975 - loss: 0.0097 - val_accuracy: 0.9552 - val_loss: 0.1908
Epoch 7/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 142s 394ms/step - accuracy: 0.9985 - loss: 0.0046 - val_accuracy: 0.9405 - val_loss: 0.1916
Epoch 8/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 142s 394ms/step - accuracy: 0.9